# Data Science z Python 3.10. Współczynnik uczenia
## 🇬🇧 Data Science with Python 3.10. Learning rate
#### 👨‍🏫 PhD Wojciech Oronowicz-Jaśkowiak
#### 🤖 https://github.com/aipogodzinach

### 1. Import bibliotek
#### 🇬🇧 Import libraries

In [ ]:
!pip install fastbook
import fastbook
from fastbook import *
from fastai.vision.widgets import *
from IPython.core.pylabtools import figsize

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataset_x4')
path = Path('/content/drive/My Drive/dataset_x4')

### 2. Trenowanie modelu

In [ ]:
classes = ['gitary', 'skrzypce']

In [ ]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(max_rotate=2.0)
)
dls = data.dataloaders(path, bs = 32, num_workers = 0)

In [ ]:
dls.valid.show_batch(max_n=12, nrows=4)

In [ ]:
learn = vision_learner(dls, resnet152, metrics=accuracy)

### 3. Współczynnik uczenia i early stopping w fastai
 - https://docs.fast.ai/tutorial.vision.html

#### 🇬🇧 Learning rate and early stopping in fastai
 - https://docs.fast.ai/tutorial.vision.html

In [ ]:
learn.fit(2)
learn.save('stage-1')
learn.lr_find()
#learn.fit(5, 5e-2)
#learn.save('stage-2')

In [ ]:
learn.fit(5, 10e-4)
learn.save('stage-2')

### 📚 Zadanie
##### 1. Przeprowadź trening sieci neuronowej przy wykorzystaniu early stoppingu (10 epok, patience = 5), zapisz model jako „stage-1”, wygeneruj macierz pomyłek (confusion matrix), oblicz wielkość F-1.
##### 2. Przeprowadź trening sieci neuronowej przy wykorzystaniu early stoppingu (10 epok, patience = 5), zapisz model jako „stage-2”, wygeneruj macierz pomyłek (confusion matrix), oblicz wielkość F-1, ale tym razem przeprowadź uczenie z wykorzystaniem optymalnego learning rate.
##### 3. Porównaj uzyskane wyniki. Który model jest lepszy?
###### Notatka: Implementacja early stopping w fastai jest prosta. Wystarczy, że do kodu związanego z trenowaniem modelu obok liczby epok, optymalnego zakresu learning rate, wprowadzisz nową zmienną cbs, która przyjmie EarlyStoppingCallback oraz SaveModelCallback.

### 🇬🇧 Task
##### 1. Perform neural network training using early stopping (10 epochs, patience = 5), save the model as "stage-1", generate a confusion matrix, calculate the value of F-1.
##### 2. Perform neural network training using early stopping (10 epochs, patience = 5), save the model as "stage-2", generate a confusion matrix, calculate the value of F-1, but this time using the optimal learning rate.
##### 3. Compare the obtained results. Which model is better?
###### Note: Implementation of early stopping in fastai library is simple. All you need to do is to enter a new cbs variable, which will adopt EarlyStoppingCallback and SaveModelCallback, to the code related to training the model, next to the number of epochs and the optimal learning rate range.

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
c1 = EarlyStoppingCallback(patience=5, monitor='valid_loss')
c2 = SaveModelCallback(monitor='valid_loss')

In [ ]:
learn.fit(10, cbs=[c1,c2])
learn.save('stage-1')
learn.plot_confusion_matrix()

In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit(10, cbs=EarlyStoppingCallback(patience=5))

learn.save('stage-2')
ClassificationInterpretation.from_learner.plot_confusion_matrix()